In [ ]:
import numpy as np
import pandas as pd
import gc


train_X = pd.read_csv("train_X_Final_order2.csv",low_memory=True)
train_Y = pd.read_csv("train_Y_Final_order2.csv",low_memory=True)
test = pd.read_csv("test_final_order2.csv",index_col='ncodpers')
test.fillna(0,inplace=True)

In [ ]:
####### INSERT LABEL ENCODER HERE  #########
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
strings = [col for col in train_X.columns if train_X[col].dtype == object]
#test.columns=[i+'_0' for i in test.columns]

fit_col = []
for i in range(len(strings)):
    print(strings[i],list((train_X[strings[i]].unique())),list((test[strings[i]].unique())))
    test[strings[i]]=test[strings[i]].astype(str)
    train_X[strings[i]]=train_X[strings[i]].astype(str)
    fit_col.append(le.fit((train_X[strings[i]].append(test[strings[i]])).astype(str)))
    train_X[strings[i]] = fit_col[i].transform(train_X[strings[i]])
    test[strings[i]] = fit_col[i].transform(test[strings[i]])

In [ ]:
train_Y=train_Y.astype(bool)
train_X=train_X.astype(np.int32)
test=test.astype(np.int32)

In [ ]:
#added products
train_X_prod=[col for col in train_X.columns if 'ult1_1' in col]
train_Y=train_Y.astype(int)-train_X.loc[:,train_X_prod]
train_Y[train_Y < 0] = 0
train_Y=train_Y.astype(bool)
train_Y.sum()

In [ ]:
"""added products"""
train_X_prod_prev=[col for col in train_X.columns if 'ult1_0' in col]
train_X.loc[:,train_X_prod]=train_X.loc[:,train_X_prod]-train_X.loc[:,train_X_prod_prev].values
train_X[train_X<0] = 0
train_X.loc[:,train_X_prod].sum()


In [ ]:
#remove rows with all 0s
Ynot1 = (train_Y == 0).all(axis=1)
train_Y=train_Y[~Ynot1]
train_X=train_X[~Ynot1]
train_Y.drop([col for col, val in train_Y.sum().iteritems() if val < 5000], axis=1, inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.ensemble import GradientBoostingClassifier 
from sklearn.multioutput import MultiOutputClassifier 


X_train, X_test, y_train, y_test = train_test_split(train_X, train_Y, 
                                                    test_size=0.06, train_size=0.24, 
                                                    random_state=40)

#clf = DecisionTreeClassifier(random_state=40, criterion='gini',max_depth=10)

clf = MultiOutputClassifier(GradientBoostingClassifier(n_estimators=200, learning_rate=0.1, max_depth=4,
                                                       random_state=40),n_jobs=8)

#clf.fit(train_X,train_Y)
clf.fit(X_train,y_train)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score,average_precision_score

y_pred = clf.predict(X_test)
print('acc',accuracy_score(y_pred, y_test),'prec',precision_score(y_test,y_pred,average='micro'))

#(random_state=40, criterion='gini',max_depth=5) -> acc 0.7520150492974298 prec 0.8042921809776741
#(random_state=40, criterion='gini',max_depth=8) -> 
#(random_state=40, criterion='gini',max_depth=10) -> 
#(random_state=40, criterion='gini',max_depth=12) -> 
#(random_state=40, criterion='gini',max_depth=15) -> acc 0.9238697615460784 prec 0.9674549120238518
#(random_state=40, criterion='gini',max_depth=18) -> acc 0.9282355411543959 prec 0.9704043581751023
#(random_state=40, criterion='gini',max_depth=20) -> acc 0.9280332656513078 prec 0.9703055805614199

In [ ]:
importance = clf.feature_importances_
# summarize feature importance

ind=[]
val=[]
for i,v in enumerate(importance):
    #imp.append(pd.Series([v],index=[train_X.columns[i][:-2]]))
    ind.append(train_X.columns[i][:-2])
    val.append(v)
    #print(pd.Series([v],index=[train_X.columns[i][:-2]]))
	#print('Feature:',train_X.columns[i], 'Score: %.5f' % (v))
imp=pd.Series(val,index=ind,name='importance',dtype=float)
imp

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.bar(imp.index, imp, color='green')
plt.xlabel("Variable")
plt.ylabel("Feature Importance")
plt.title("Importance")
plt.xticks(imp.index, rotation=90)
plt.show()

In [ ]:
y_pred = clf.predict_proba(test)
y_pred_df = pd.DataFrame(columns=train_Y.columns,index=test.index)
for i,j in zip(y_pred,train_Y.columns):
    y_pred_df[j]=i[:,1]
    
y_pred_df.columns=[i[:-2] for i in y_pred_df.columns]

In [ ]:
train_prev = pd.read_csv("train_2016052018.csv",index_col='ncodpers')
train_prev.columns = [col[:-1] if '_201' not in col else col[:-11] for col in train_prev.columns]
y_pred_df=y_pred_df.sort_index()
train_prev=train_prev.sort_index()
train_prev = train_prev[train_prev.index.isin(y_pred_df.index)]
print(train_prev.shape,y_pred_df.shape)
output = y_pred_df-train_prev
output[output < 0] = 0
output.sum()


In [ ]:
output = output.apply(lambda x: (x.nlargest(n=7,keep="first").index), axis=1)

In [ ]:
outfile = output.apply(lambda x: ' '.join(x[x!='']))

In [ ]:
outfile.name = 'added_products'
outfile.index.rename('ncodpers',inplace=True)
outfile.to_csv('outfile_gb4_order2.csv',sep=',',index=True)

In [ ]:
outfile